In [13]:
import ipywidgets as widgets
from ipywidgets import interact

from energy_check import energy_check
from conf_checker import conf_checker
from out_checker import out_checker
from energy_check import energy_check
from baseline_energy import baseline_energy
from magnetic_check import magnetic_check
from visualization_energy_difference import visualization_energy_difference
from visualization_lattice import visualization_lattice
from lattice_check import lattice_check
from visualization import visualization
from show_structure import show_structure
from magnetic_plot import magnetic_plot
from voltage import voltage
from voltage_visualization import voltage_visualization

#output = widgets.Output()
#plot_output = widgets.Output()
#plot1_output = widgets.Output()

def common_filtering(model, structure, li_concentration, functional_list,U_value,what,conf,visualize,figure,configuration,energy_baseline):
  
    output.clear_output()
    #plot_output.clear_output()
    #plot1_output.clear_output()
    
    if (model == 'None'):
        common_filter = 'Please chose a material.'
    elif (len(conf_checker(model,li_concentration,structure)) <= conf):
        common_filter = "This material in combination with this structure does not have this configuration. Try a lower number."
    elif (model != 'Li2MnO3') & (structure == 'O1'):
        common_filter = 'The O1-stacking is only an option for Li2MnO3.'
    elif (what == 'Final Energy') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,energy] = energy_check(model,structure,li_concentration,functional_list,U_value,conf,configuration)
        common_filter = tabulate(energy, headers=header, tablefmt="latex",floatfmt=(".1f",".2f"))
    elif (what == 'out - required accuracy') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        common_filter = out_checker(model,li_concentration,structure,functional_list,U_value,conf,configuration)
    elif (what == 'Magnetic moments') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,results] = magnetic_check(model,structure,li_concentration,functional_list,U_value,conf)
        common_filter = tabulate(results, headers=header, tablefmt="latex", floatfmt=".2f")
    elif (what == 'lattice parameters') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,lattice_parameters_length] = lattice_check(model,structure,li_concentration,functional_list,U_value,conf)
        common_filter = tabulate(lattice_parameters_length, headers=header, tablefmt="latex", floatfmt=(".1f",".5f"))
    elif (what == 'lattice angles') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,lattice_parameters_angles] = lattice_angles_check(model,structure,li_concentration,functional_list,U_value,conf)
        common_filter = tabulate(lattice_parameters_angles, headers=header, tablefmt="latex", floatfmt=(".1f",".5f"))
    elif (what == 'voltage') & (model != 'None') & (len(conf_checker(model,li_concentration,structure)) > conf):
        [header,result] = voltage(model,structure,li_concentration,functional_list,U_value,conf,configuration)
        common_filter = tabulate(result, headers=header, tablefmt="latex", floatfmt=(".1f",".2f"))
    
    with output:
        if (what == 'out - required accuracy'):
            display(common_filter)
        else:
            print(common_filter)
            
    with plot_output:
        if (len(conf_checker(model,li_concentration,structure)) <= conf):
            print("This material in combination with this structure does not have this configuration. Try a lower number.")
        elif (U_value == (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)) & (functional_list == 'pbeU'):
            print('Cannot visualize multiple files. Change the U-value setting.')
        elif (model == 'None'):
            print('Please chose a material.')
        elif (visualize != True):
            print('If you want to viualize a structure check the Visualize Structure checkbox.')
        elif (visualize == True):
            visualization(model,li_concentration,structure,functional_list,U_value,conf)
              
    with plot1_output: 
        if (model == 'None'):
            print('Please chose a material.')
        elif (len(conf_checker(model,li_concentration,structure)) <= conf):
            print("This material in combination with this structure does not have this configuration. Try a lower number.")
        elif (figure != True):
            print('If you want to viualize a structure check the Figures checkbox.')
        elif (U_value != (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)):
            print('The figures are only interesting for all U-values.')
        elif (what == 'voltage') & (figure == True) & (U_value != (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)):
            print('The figures are only interesting for all U-values.')
        elif (what == 'lattice parameters') & (figure == True):
            visualization_lattice(model,li_concentration,structure,functional_list,U_value,conf)
        elif (what == 'lattice angles') & (figure == True):
            visualization_lattice(model,li_concentration,structure,functional_list,U_value,conf)
        elif (what == 'Final Energy') & (figure == True):
            visualization_energy_difference(model,li_concentration,structure,functional_list,U_value,conf,configuration,energy_baseline)
        elif (what == 'Magnetic moments') & (figure == True):
            magnetic_plot(model,li_concentration,structure,functional_list,U_value,conf)
        elif (what == 'voltage') & (figure == True) & (U_value == (0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6)):
            voltage_visualization(model,structure,li_concentration,functional_list,U_value,conf,configuration) 
            
interact(common_filtering,         
# All the defenitions of the dropdown menu's
model = widgets.Dropdown(
    options=["None","Li2MnO3", "Li2IrO3","Li2MnSnO3"],
    description='Material:',
    disabled=False,
),
         
structure = widgets.Dropdown(
    options=['prim','conv','O1'],
    description='Cell: (only for Li2MnO3)',
    disabled=False,
),

li_concentration = widgets.Dropdown(
    options=["0.5","2"],
    description='Li conc:',
    disabled=False,
),

functional_list = widgets.Dropdown(
    options=["pbeU", "scan" , "hse",("pbeU", "scan"),("pbeU", "scan" , "hse")],
    description='Functional:',
    disabled=False,
),

u_values = ((0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6),0, 0.5, 1, 1.5, 2, 2.5, 3, 3.5, 3.9, 4, 4.5, 5, 5.5, 6),
U_value = widgets.Dropdown(
    options=u_values, 
    description='U-value: (only for pbeU)',
    disabled=False,
),

what = widgets.Dropdown(
    options= ["Final Energy", "Magnetic moments","lattice parameters","lattice angles","voltage","out - required accuracy"],
    description='What?:',
    disabled=False,
),

conf = widgets.Dropdown(
    #options= conf_checker(models,li_concentrations,structures),
    options = (0,1,2,3,4,5,6,7,8,9,10),
    description='Conf:',
    disabled=False,
),

energy_baseline = widgets.Dropdown(
    options= ["Discharged", "Absolute","Minimum"],
    description='Baseline for Final Energy:',
    disabled=False,
),

configuration = widgets.Checkbox(
           description='Conf: All.',),

visualize = widgets.Checkbox(
           description='Unlock: Visualize structure.',),

figure = widgets.Checkbox(
           description='Unlock: Figures.',)
)
output

interactive(children=(Dropdown(description='Material:', options=('None', 'Li2MnO3', 'Li2IrO3', 'Li2MnSnO3'), v…

Output(outputs=({'output_type': 'stream', 'text': 'Please chose a material.\n', 'name': 'stdout'},))

In [35]:
def running(test):
    if test == 10:
        result = testing(test) + 1
    else:
        result = tester(test)
    return result  

interact(running, test = widgets.Dropdown(
    options=[10,20],
    description='Material:',
    disabled=False,
))

interactive(children=(Dropdown(description='Material:', options=(10, 20), value=10), Output()), _dom_classes=(…

<function __main__.running(test)>

In [ ]:
# %load tester.py
def tester(x):
    result = x + 10
    return result